In [1]:
'''
  Reference : https://github.com/roomylee/cnn-relation-extraction
'''

import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm

## Parameters

In [2]:
class Config:
    # Data loading params
    max_sentence_length = 90
    dev_sample_percentage = 0.1
    
    # Embeddings
    embedding_path = ''
    text_embedding_dim = 300
    pos_embedding_dim = 50
    
    # CNN
    filter_sizes = '2,3,4,5'
    num_filters = 128
    dropout_keep_prob = 0.5
    l2_reg_lambda = 1e-5
    
    # Training parameters
    batch_size = 20
    num_epochs = 100
    display_every = 500
    evaluate_every = 1000
    num_checkpoints = 5
    learning_rate = 1.0
    decay_rate = 0.9
    
    # Testing parameters
    checkpoint_dir = ''

    labels_count = 19
    class2label = {'Other': 0,
               'Message-Topic(e1,e2)': 1, 'Message-Topic(e2,e1)': 2,
               'Product-Producer(e1,e2)': 3, 'Product-Producer(e2,e1)': 4,
               'Instrument-Agency(e1,e2)': 5, 'Instrument-Agency(e2,e1)': 6,
               'Entity-Destination(e1,e2)': 7, 'Entity-Destination(e2,e1)': 8,
               'Cause-Effect(e1,e2)': 9, 'Cause-Effect(e2,e1)': 10,
               'Component-Whole(e1,e2)': 11, 'Component-Whole(e2,e1)': 12,
               'Entity-Origin(e1,e2)': 13, 'Entity-Origin(e2,e1)': 14,
               'Member-Collection(e1,e2)': 15, 'Member-Collection(e2,e1)': 16,
               'Content-Container(e1,e2)': 17, 'Content-Container(e2,e1)': 18}

    label2class = {0: 'Other',
                   1: 'Message-Topic(e1,e2)', 2: 'Message-Topic(e2,e1)',
                   3: 'Product-Producer(e1,e2)', 4: 'Product-Producer(e2,e1)',
                   5: 'Instrument-Agency(e1,e2)', 6: 'Instrument-Agency(e2,e1)',
                   7: 'Entity-Destination(e1,e2)', 8: 'Entity-Destination(e2,e1)',
                   9: 'Cause-Effect(e1,e2)', 10: 'Cause-Effect(e2,e1)',
                   11: 'Component-Whole(e1,e2)', 12: 'Component-Whole(e2,e1)',
                   13: 'Entity-Origin(e1,e2)', 14: 'Entity-Origin(e2,e1)',
                   15: 'Member-Collection(e1,e2)', 16: 'Member-Collection(e2,e1)',
                   17: 'Content-Container(e1,e2)', 18: 'Content-Container(e2,e1)'}

## Dataset 

Load Relation Extraction dataset of SemEval2010 task8

In [3]:
import nltk
import re
import os

class Dataset:
    def clean_str(self, text):
        text = text.lower()
        # Clean the text
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"that's", "that is ", text)
        text = re.sub(r"there's", "there is ", text)
        text = re.sub(r"it's", "it is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "can not ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"\-", " - ", text)
        text = re.sub(r"\=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)

        return text.strip()

    def load_data_and_labels(self, path):
        # Data Format
        # 1\t"The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>."
        # Component-Whole(e2,e1)
        # Comment: Not a collection: there is structure here, organisation.
        # 
        # 2\t"The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord."
        # Other
        # Comment:
        # 
        data = []
        lines = [line.strip() for line in open(path)]
        max_sentence_length = 0
        for idx in range(0, len(lines), 4):
            id = lines[idx].split("\t")[0]
            
            # Sentence
            sentence = lines[idx].split("\t")[1][1:-1]
            sentence = sentence.replace('<e1>', ' _e11_ ')
            sentence = sentence.replace('</e1>', ' _e12_ ')
            sentence = sentence.replace('<e2>', ' _e21_ ')
            sentence = sentence.replace('</e2>', ' _e22_ ')

            sentence = self.clean_str(sentence)
            tokens = nltk.word_tokenize(sentence)
            sentence = " ".join(tokens)
            
            # Max Sentence Length
            if max_sentence_length < len(tokens):
                max_sentence_length = len(tokens)
                
            # e1, e2 position
            e1 = tokens.index("e12") - 1
            e2 = tokens.index("e22") - 1
            
            # Relative Position
            pos1 = ""
            pos2 = ""
            for word_idx in range(len(tokens)):
                pos1 += str((Config.max_sentence_length - 1) + word_idx - e1) + " "
                pos2 += str((Config.max_sentence_length - 1) + word_idx - e2) + " "
                
            # Label
            relation = lines[idx + 1]
            label = Config.class2label[relation]
            data.append([id, sentence, e1, e2, pos1, pos2, relation, label])

        print(path)
        print("max sentence length = {}\n".format(max_sentence_length))

        df = pd.DataFrame(data=data, columns=["id", "sentence", "e1", "e2", 'pos1', 'pos2', 'relation', 'label'])
        return df
    
    def download_and_load_datasets(self):
        dataset = tf.keras.utils.get_file(
          fname="SemEval2010_task8_all_data.zip", 
          origin="https://s3.ap-northeast-2.amazonaws.com/bowbowbow-storage/dataset/SemEval2010_task8_all_data.zip", 
          extract=True)
        
        train_file = 'SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT'
        test_file = 'SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT'
        
        train_df = self.load_data_and_labels(os.path.join(os.path.dirname(dataset), train_file))
        test_df = self.load_data_and_labels(os.path.join(os.path.dirname(dataset), test_file))
        return train_df, test_df

dataset = Dataset()
train_df, test_df = dataset.download_and_load_datasets()
train_df.head()

/home/seungwon/.keras/datasets/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT
max sentence length = 89

/home/seungwon/.keras/datasets/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT
max sentence length = 68



,id,sentence,e1,e2,pos1,pos2,relation,label
0,1,the system as described above has its greatest...,13,18,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 8...,"Component-Whole(e2,e1)",12
1,2,the e11 child e12 was carefully wrapped and bo...,2,12,87 88 89 90 91 92 93 94 95 96 97 98 99 100 101...,77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 9...,Other,0
2,3,the e11 author e12 of a keygen uses a e21 disa...,2,10,87 88 89 90 91 92 93 94 95 96 97 98 99 100 101...,79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 9...,"Instrument-Agency(e2,e1)",6
3,4,a misty e11 ridge e12 uprises from the e21 sur...,3,9,86 87 88 89 90 91 92 93 94 95 96,80 81 82 83 84 85 86 87 88 89 90,Other,0
4,5,the e11 student e12 e21 association e22 is the...,2,5,87 88 89 90 91 92 93 94 95 96 97 98 99 100 101...,84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 9...,"Member-Collection(e1,e2)",15


## CNN Model

In [ ]:
class TextCNN:
    def __init__(self, 
               sequence_length, 
               num_classes, 
               text_vocab_size, 
               text_embedding_size, 
               pos_vocab_size, 
               pos_embedding_size, 
               filter_sizes,
               num_filters, 
               l2_reg_lambda=0.0):
        
        self.input_text = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_text')
        self.input_p1 = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_p1')
        self.input_p2 = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_p2')
        self.input_y = tf.placeholder(tf.float32, shape=[None, num_classes], name='input_y')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
        
        # Embedding layer
        with tf.variable_scope('text-embedding'):
            self.W_text = tf.Variable(tf.random_uniform([text_vocab_size, text_embedding_size], -0.25, 0.25), name='W_text')
            self.text_embedded_chars = tf.nn.embedding_lookup(self.W_text, self.input_text)
            self.text_embedded_chars_expanded = tf.expand_dims(self.text_embedded_chars, -1)
            
        with tf.variable_scope('position-embedding'):
            self.W_pos = tf.get_variable('W_pos', [pos_vocab_size, pos_embedding_size], initializer=tf.keras.initializers.glorot_normal())
            self.p1_embedded_chars = tf.nn.embedding_lookup(self.W_pos, self.input_p1)
            self.p2_embedded_chars = tf.nn.embedding_lookup(self.W_pos, self.input_p2)
            self.p1_embedded_chars_expanded = tf.expand_dims(self.p1_embedded_chars, -1)
            self.p2_embedded_chars_expanded = tf.expand_dims(self.p2_embedded_chars, -1)
        
        self.embedded_chars_expanded = tf.concat([self.text_embedded_chars_expanded,
                                                  self.p1_embedded_chars_expanded,
                                                  self.p2_embedded_chars_expanded], 2)
        
        _embedding_size = text_embedding_size + 2*pos_embedding_size
        
        # convolution + maxpool layer
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope('conv-maxpool-{}'.format(filter_size)):
                conv = tf.layers.conv2d(self.embedded_chars_expanded, 
                                       num_filters, 
                                       [filter_size, _embedding_size], 
                                       kernel_initializer=tf.keras.initializers.glorot_normal(),
                                       activation=tf.nn.relu,
                                       name='conv')
                pooled = tf.nn.max_pool(conv, 
                                        ksize=[1, sequence_length - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1],
                                        padding='VALID',
                                        name='pool'
                                       )
                pooled_outputs.append(pooled)
                
        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        
        self.h_pool = tf.concat(pooled_outputs, 3) # [batch_size, 1, 1, num_filters]
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        
        # Add dropout
        with tf.variable_scope('dropout'):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
        
        # Final scores and predictions
        with tf.variable_scope('output'):
            self.logits = tf.layers.dense(self.h_drop, num_classes, kernel_initializer=tf.keras.initializers.glorot_normal())
            self.predictions = tf.argmax(self.logits, 1, name='predictions')
        
        # Calculate mean corss-entropy loss
        with tf.variable_scope('loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.input_y)
            self.l2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * self.l2
        
        # Accuracy    
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')

## Preprocessing

In [ ]:
# Build vocabulary
text_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(Config.max_sentence_length)
x = np.array(list(text_vocab_processor.fit_transform(train_df['sentence'])))
y = np.array([np.eye(Config.labels_count)[label] for label in train_df['label']]) # One-hot encoding 

print('Text Vocabulary Size {}'.format(len(text_vocab_processor.vocabulary_)))
print('X = {}'.format(x.shape))
print('Y = {}'.format(y.shape))

# Example: pos1[3] = '95 96 97 98 99 100 101 999 999 999 ... 999' (without offset: [-2 -1  0  1  2   3   4 999 999 999 ... 999])
# =>
# [11 12 13 14 15  16  21  17  17  17 ...  17]
pos_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(Config.max_sentence_length)
pos_vocab_processor.fit(train_df['pos1'] + train_df['pos2'])

p1 = np.array(list(pos_vocab_processor.transform(train_df['pos1'])))
p2 = np.array(list(pos_vocab_processor.transform(train_df['pos2'])))

print('Position Vocabulary Size {}'.format(len(pos_vocab_processor.vocabulary_)))
print('position1 = {}'.format(p1.shape))
print('position2 = {}'.format(p2.shape))

# Randomly shuffle data to split into train and dev
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled, p1_shuffled, p2_shuffled, y_shuffled = x[shuffle_indices], p1[shuffle_indices], p2[shuffle_indices], y[shuffle_indices]

# Split train/dev set
dev_sample_index = -1 * int(Config.dev_sample_percentage*float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:] 
p1_train, p1_dev = p1_shuffled[:dev_sample_index], p1_shuffled[dev_sample_index:] 
p2_train, p2_dev = p2_shuffled[:dev_sample_index], p2_shuffled[dev_sample_index:] 
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:] 
print("Train/Dev split: {:d}/{:d}\n".format(len(y_train), len(y_dev)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Text Vocabulary Size 19151
X = (8000, 90)
Y = (8000, 19)
Position Vocabulary Size 162
position1 = (8000, 90)
position2 = (8000, 90)
Train/Dev split: 7200/800



## Function for train

In [ ]:
# Pre-trained word2vec
def load_word2vec(embedding_dim, vocab):
    download_path = tf.keras.utils.get_file(
      fname="GoogleNews-vectors-negative300.bin.gz", 
      origin="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", 
      extract=True)
    
    embedding_path = os.path.join(os.path.dirname(download_path), 'GoogleNews-vectors-negative300.bin')
    if not os.path.exists(embedding_path):
        print('unzip :', embedding_path)
        import gzip
        import shutil
        with gzip.open('{}.gz'.format(embedding_path), 'rb') as f_in:
            with open(embedding_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    print('embedding_path :', embedding_path)

    # initial matrix with random uniform
    initW = np.random.randn(len(vocab.vocabulary_), embedding_dim).astype(np.float32) / np.sqrt(len(vocab.vocabulary_))
    # load any vectors from the word2vec
    print("Load word2vec file {0}".format(embedding_path))
    with open(embedding_path, "rb") as f:
        header = f.readline()
        vocab_size, layer_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1).decode('latin-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            idx = vocab.vocabulary_.get(word)
            if idx != 0:
                initW[idx] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return initW

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

## Train

In [ ]:
import datetime
import time

from sklearn.metrics import f1_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():
    cnn = TextCNN(
        sequence_length=x_train.shape[1],
        num_classes=y_train.shape[1],
        text_vocab_size=len(text_vocab_processor.vocabulary_),
        text_embedding_size=Config.text_embedding_dim,
        pos_vocab_size=len(pos_vocab_processor.vocabulary_),
        pos_embedding_size=Config.pos_embedding_dim,
        filter_sizes=list(map(int, Config.filter_sizes.split(','))),
        num_filters=Config.num_filters,
        l2_reg_lambda=Config.l2_reg_lambda
    )
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    optimizer = tf.train.AdadeltaOptimizer(Config.learning_rate, Config.decay_rate, 1e-6)
    gvs = optimizer.compute_gradients(cnn.loss)
    capped_gvs = [(tf.clip_by_value(grad, -1.0, 1.0), var) for grad, var in gvs]
    train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)
    
    # Output directory for models and summary
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "23.runs", timestamp))
    print("Writing to {}\n".format(out_dir))

    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", cnn.loss)
    acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
    
    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary, acc_summary])
    train_summary_dir = os.path.join(out_dir, "summaries", "train")
    train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

    # Dev summaries
    dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
    dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
    dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

    # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
    checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
    checkpoint_prefix = os.path.join(checkpoint_dir, "model")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=Config.num_checkpoints)
    
    sess.run(tf.global_variables_initializer())

    pretrain_W = load_word2vec(Config.text_embedding_dim, text_vocab_processor)
    sess.run(cnn.W_text.assign(pretrain_W))
    print("Success to load pre-trained word2vec model!\n")
    
    # Generate batches
    batches = batch_iter(list(zip(x_train, p1_train, p2_train, y_train)),
                                      Config.batch_size, 
                                      Config.num_epochs)
    
    # Training loop. For each batch...
    best_f1 = 0.0  # For save checkpoint(model)
    for batch in batches:
        x_batch, p1_batch, p2_batch, y_batch = zip(*batch)
        # Train
        feed_dict = {
            cnn.input_text: x_batch,
            cnn.input_p1: p1_batch,
            cnn.input_p2: p2_batch,
            cnn.input_y: y_batch,
            cnn.dropout_keep_prob: Config.dropout_keep_prob
        }
        _, step, summaries, loss, accuracy = sess.run(
            [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy], feed_dict)
        train_summary_writer.add_summary(summaries, step)

        # Training log display
        if step % Config.display_every == 0:
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

        # Evaluation
        if step % Config.evaluate_every == 0:
            print("\nEvaluation:")
            feed_dict = {
                cnn.input_text: x_dev,
                cnn.input_p1: p1_dev,
                cnn.input_p2: p2_dev,
                cnn.input_y: y_dev,
                cnn.dropout_keep_prob: 1.0
            }
            summaries, loss, accuracy, predictions = sess.run(
                [dev_summary_op, cnn.loss, cnn.accuracy, cnn.predictions], feed_dict)
            dev_summary_writer.add_summary(summaries, step)

            time_str = datetime.datetime.now().isoformat()
            f1 = f1_score(np.argmax(y_dev, axis=1), predictions, labels=np.array(range(1, 19)), average="macro")
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            print("[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): {:g}\n".format(f1))

            # Model checkpoint
            if best_f1 < f1:
                best_f1 = f1
                path = saver.save(sess, checkpoint_prefix + "-{:.3g}".format(best_f1), global_step=step)
                print("Saved model checkpoint to {}\n".format(path))



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Writing to /home/seungwon/project/tf-notes/23.runs/1557233711

embedding_path : /home/seungwon/.keras/datasets/GoogleNews-vectors-negative300.bin
Load word2vec file /home/seungwon/.keras/datasets/GoogleNews-vectors-negative300.bin


/home/seungwon/project/tf-notes/venv/lib/python3.5/site-packages/ipykernel_launcher.py:38: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Success to load pre-trained word2vec model!

2019-05-07T21:55:37.014310: step 500, loss 1.38153, acc 0.8
2019-05-07T21:55:44.275194: step 1000, loss 1.54936, acc 0.75

Evaluation:
2019-05-07T21:55:44.773806: step 1000, loss 1.63837, acc 0.7575
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.732765

Saved model checkpoint to /home/seungwon/project/tf-notes/23.runs/1557233711/checkpoints/model-0.733-1000

2019-05-07T21:55:52.238061: step 1500, loss 1.01738, acc 0.9
2019-05-07T21:55:59.522175: step 2000, loss 1.02222, acc 0.95

Evaluation:
2019-05-07T21:55:59.598847: step 2000, loss 1.65998, acc 0.77125
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.74938

Saved model checkpoint to /home/seungwon/project/tf-notes/23.runs/1557233711/checkpoints/model-0.749-2000

2019-05-07T21:56:07.002339: step 2500, loss 0.890682, acc 1
2019-05-07T21:56:14.277781: step 3000, loss 0.839589, acc 1

Evaluation:
2019-05-07T21:56:14.355677: step 3000, loss 1.67266, ac

## Tensorboard

```
tensorboard --logdir=./23.runs --host 0.0.0.0
```